## optimisation algos

In [1]:
# Importer les bibliothèques nécessaires

import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, confusion_matrix,accuracy_score
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import missingno as msno
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import seaborn as sns
import imblearn as imb
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from sklearn.metrics import auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
from imblearn import FunctionSampler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn
import openpyxl
import xlwt


In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

In [3]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base=pd.read_csv("/content/drive/MyDrive/Notebooks/P7/basep7.csv")
        
except ModuleNotFoundError :
    
    base = pd.read_csv("basep7.csv")
    
base = base.drop( columns = ['Unnamed: 0'])

In [4]:
# one_hot_encoder classique pour les non numériques
def one_hot_encoder(base, nan_as_category = True):
    original_columns = list(base.columns)
    categorical_columns = [col for col in base.columns if base[col].dtype == 'object']
    base2 = pd.get_dummies(base, columns= categorical_columns, dummy_na= True)
    new_columns = [c for c in base.columns if c not in original_columns]
    return base2
print (base.shape)
base2 =one_hot_encoder(base)
print (base2.shape)
base = base2

(307511, 122)
(307511, 262)


In [5]:
# Remplacer les valeurs manquantes par la moyenne de la colonne
base = base.fillna(base.mean())
#base= base.sample(500)

In [6]:
# Séparer les variables explicatives (X) et la variable cible (y)
X = base.drop("TARGET", axis=1)
y = base["TARGET"]



In [7]:
# scaler 
col_names=X.select_dtypes(include='number').columns.tolist()
features = X[col_names]
scaler = StandardScaler().fit(features.values)
features_scale = scaler.transform(features.values)
X[col_names] = features_scale
print (X.shape) 
del features
del features_scale

(307511, 261)


In [8]:
def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

In [9]:
debut = time.time()

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 



In [11]:
#X_resampled, y_resampled = SMOTETomek(random_state=22).fit_resample(X_train,y_train)

In [12]:
def comp_algos(X,y,algos,params,cvi,scorer,sampling) : 

    
    roc_aucs = []
    confusions = []
    Report_df = pd.DataFrame(columns=['Algorithme', 'score nom ','score']) 
    Append = []

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    
    
    
    
    
    for i in range(len(algos)): 
        with mlflow.start_run(experiment_id=experiment_id):          
                   
            #faire boucle sur le sampling 
            for s in range(len(sampling)): 
                with mlflow.start_run(run_name=str(algos[i])+"Child Run", nested=True):
                #with mlflow.start_run(run_name="Child Run", nested=True):
            
                    # Apply sampling            
                    debut = time.time()
                    X_resampled, y_resampled = sampling[s].fit_resample(X_train,y_train)

# Créer un objet GridSearchCV qui va comparer les performances des algorithmes sur les données en utilisant le score roc_auc comme critère d'évaluation 
                    print (algos[i], params[i] )          
                    grid = GridSearchCV(estimator=algos[i], param_grid=params[i], scoring=scorer, cv=cvi, refit= True )
                    grid.fit(X_resampled,y_resampled)
                    del X_resampled
                    del y_resampled
            
                    """# Afficher le meilleur score et le meilleur algorithme trouvé par GridSearchCV 
                    print("Meilleur score:", grid.best_score_)
                    print("Meilleur algorithme:", grid.best_estimator_)
                    #print ( "cv results ;", grid.cv_results_ ) 
                    print ( "best_params_",grid.best_params_)
                    print ("best_index_",grid.best_index_)
                    print ("scorer_",grid.scorer_)
                    print ("n_splits_",grid.n_splits_)
                    print ("sampling method",sampling[s])"""
                        
                    cv_result= pd.DataFrame(grid.cv_results_)
                    display ( cv_result )

# Prédire les classes pour chaque algorithme 
                    
                    pred_y_train = grid.predict(X_train)
                    score = accuracy_score(y_train,pred_y_train)
                    print ('accuracy train',score) 
          
                    roc_auc_train = roc_auc_score(y_train, pred_y_train)
                    #print ( 'roc_auc_train',roc_auc_train)
                    print(f'roc_auc_train: {roc_auc_train:.1%}')

                    pred_y_train_proba=grid.predict_proba(X_train)[:,1]
                    roc_auc_discret_train = roc_auc_score(y_train, pred_y_train_proba)
                    print( "Roc auc train predict proba",roc_auc_discret_train)
        
                    pred_y_test = grid.predict(X_test)
                    score = accuracy_score(y_test,pred_y_test)
                    print ('accuracy test',score)
          
                    roc_auc_test = roc_auc_score(y_test, pred_y_test)
                    print ( 'roc_auc_ test ',roc_auc_test)

                    pred_y_test_proba=grid.predict_proba(X_test)[:,1]
                    roc_auc_discret_test= roc_auc_score(y_test, pred_y_test_proba)
                    print( "Roc auc score test predict proba",roc_auc_discret_test)

# Calculer la matrice de confusion pour chaque algorithme

                    matrice_confusion = confusion_matrix(y_true=y_train,y_pred=pred_y_train)
                    tn, fp, fn, tp = confusion_matrix(y_true=y_train,y_pred=pred_y_train, normalize='all').ravel()

           
# Créer un dataframe pandas qui contient les résultats de chaque algorithme
                    print(' score ', scorer,':' ,grid.best_score_)
                    duration = (time.time() - debut )/60
                    print (duration ,'minutes')
                    Append = pd.DataFrame([[ algos[i],grid.best_params_,scorer ,grid.best_score_,sampling[s],roc_auc_train,roc_auc_discret_train
                                    ,roc_auc_test,roc_auc_discret_test,tn,fp,fn,tp,duration]]
                                  ,columns=['Algorithme','Meilleur param', 'score nom ','best score','sampling','roc_auc_train'
                                            ,'Roc_auc_train_discret','roc_auc_test','roc_auc_discret_test','TN','FP','FN','TP','minutes'])
                    Report_df = pd.concat([Report_df, Append]) 
            
                                     
                    mlflow.log_param("scorer name", scorer)
                    mlflow.log_param("sampling", sampling[s])
                    mlflow.log_param("params", params[i])
                    mlflow.log_param("best_params", grid.best_params_)
                    mlflow.log_metric("best Score", grid.best_score_)
                    mlflow.log_metric("roc auc proba test", roc_auc_discret_test)
                    mlflow.log_metric("roc auc test", roc_auc_test)
                    mlflow.log_metric("roc auc proba train", roc_auc_discret_train)
                    mlflow.log_metric("roc auc train", roc_auc_train)
                    mlflow.log_metric("duree", duration)
                    
                mlflow.sklearn.log_model(algos[i], "algo_model")
                mlflow.log_param("algo", algos[i])
                mlflow.sklearn.log_model(grid.best_estimator_, "model best estimator")
                mlflow.end_run()
        
    return Report_df

In [13]:
algos = [ LogisticRegression(random_state=22),DecisionTreeClassifier(random_state=22), 
         RandomForestClassifier(random_state=22), GaussianNB(),DummyClassifier()]
# retrait de XGboost car la chaine de caracteres des parametres est limitée à 500, elle atteint 738 avec XGboost

# Définir une grille de paramètres à optimiser pour chaque algorithme
params = [
    
    {"C": [0.01, 0.1, 1, 10], "penalty": ["l2", None]}, # pour LogisticRegression ‘l1’, ‘l2’, ‘elasticnet’, None
    {"max_depth": [3, 5, 7], "min_samples_leaf": [5, 10, 15]}, # pour DecisionTreeClassifier
    {"n_estimators": [100, 200], "max_depth": [3, 5]}, # pour RandomForestClassifier 
    {} ,# pas de paramètres à optimiser pour GaussianNB 
    {"strategy": ["most_frequent","prior","stratified","uniform"]}, # pour DummyClassifier
    ]


cvi =3

"""algos = [ DecisionTreeClassifier(random_state=22)]
params = [ {"max_depth": [3], "min_samples_leaf": [5]}] # pour DecisionTreeClassifier"""

def custom_score_func(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    score = tp + tn - fp -( 10 * fn)
    #score 2 = tp + (10 * tn )- fp -(10 * fn)
    return score


# Création du scorer personnalisé
score_metier= make_scorer(custom_score_func)

scorer = score_metier

def nosamplerfunc(X, y):
    return X[:], y[:]
nosampler = FunctionSampler(func=nosamplerfunc)

sampling = [nosampler, RandomOverSampler(random_state=22),RandomUnderSampler(random_state=22),SMOTETomek(random_state=22)]

expe = "test_230422_07"
experiment_id = mlflow.create_experiment(expe)

result_metier= comp_algos(X,y,algos,params,cvi,scorer,sampling)
result_metier

result_metier.to_excel(expe+".xls")

LogisticRegression(random_state=22) {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2', None]}


C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,12.633205,2.428574,0.137490,0.034660,0.01,l2,"{'C': 0.01, 'penalty': 'l2'}",9698.0,9655.0,9754.0,9702.333333,40.532566,4
1,0.394073,0.019596,0.000000,0.000000,0.01,None,"{'C': 0.01, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,5
2,8.402229,0.964802,0.075507,0.015640,0.1,l2,"{'C': 0.1, 'penalty': 'l2'}",9736.0,9731.0,9815.0,9760.666667,38.473656,3
3,0.288562,0.032939,0.000000,0.000000,0.1,None,"{'C': 0.1, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,6
4,7.813357,0.283466,0.069324,0.011647,1,l2,"{'C': 1, 'penalty': 'l2'}",9765.0,9780.0,9788.0,9777.666667,9.533566,2
5,0.247078,0.006874,0.000000,0.000000,1,None,"{'C': 1, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,7
6,8.923591,0.228576,0.087102,0.006001,10,l2,"{'C': 10, 'penalty': 'l2'}",9787.0,9776.0,9795.0,9786.000000,7.788881,1
7,0.278552,0.040249,0.000000,0.000000,10,None,"{'C': 10, 'penalty': None}",NaN,NaN,NaN,NaN,NaN,8


accuracy train 0.9191367760398036
roc_auc_train: 50.5%
Roc auc train predict proba 0.7481604424339416
accuracy test 0.919711233598361
roc_auc_ test  0.5054754193253597
Roc auc score test predict proba 0.7494786702745164
 score  make_scorer(custom_score_func) : 9786.0
2.2895999709765116 minutes
LogisticRegression(random_state=22) {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2', None]}


C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hugues\AppData\Roaming\Python\Python310\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

...

accuracy train 0.5004999837403662
roc_auc_train: 49.8%
Roc auc train predict proba 0.5
accuracy test 0.5019592540201291
roc_auc_ test  0.5015731510684678
Roc auc score test predict proba 0.5
 score  make_scorer(custom_score_func) : -338244.6666666667
0.24061512152353923 minutes
DummyClassifier() {'strategy': ['most_frequent', 'prior', 'stratified', 'uniform']}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_strategy,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.063496,0.033391,0.002992,8.485379e-07,most_frequent,{'strategy': 'most_frequent'},-59652.0,-59652.0,-59652.0,-59652.000000,0.000000,3
1,0.038223,0.000933,0.002991,8.134202e-04,prior,{'strategy': 'prior'},-59652.0,-59652.0,-59652.0,-59652.000000,0.000000,3
2,0.038563,0.005171,0.004655,4.701905e-04,stratified,{'strategy': 'stratified'},-29102.0,-29357.0,-30004.0,-29487.666667,379.654521,1
3,0.039894,0.003734,0.002991,8.137122e-04,uniform,{'strategy': 'uniform'},-30291.0,-29601.0,-30020.0,-29970.666667,283.843071,2


accuracy train 0.4988699554486033
roc_auc_train: 49.8%
Roc auc train predict proba 0.498476549883791
accuracy test 0.4956831374079313
roc_auc_ test  0.4904032106275285
Roc auc score test predict proba 0.49655441475776746
 score  make_scorer(custom_score_func) : -29487.666666666668
0.05737514495849609 minutes
DummyClassifier() {'strategy': ['most_frequent', 'prior', 'stratified', 'uniform']}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_strategy,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,2.502022,2.446683,0.067819,0.029629,most_frequent,{'strategy': 'most_frequent'},-678366.0,-1.0,-678367.0,-452244.666667,319784.563449,3
1,0.603717,0.066127,0.039562,0.003672,prior,{'strategy': 'prior'},-678366.0,-1.0,-678367.0,-452244.666667,319784.563449,3
2,0.606711,0.067191,0.063165,0.010311,stratified,{'strategy': 'stratified'},-337015.0,-341224.0,-336655.0,-338298.000000,2074.207801,1
3,0.669210,0.064331,0.045878,0.001410,uniform,{'strategy': 'uniform'},-337905.0,-340297.0,-337999.0,-338733.666667,1106.109498,2


accuracy train 0.49934148482976165
roc_auc_train: 50.2%
Roc auc train predict proba 0.4989295269588966
accuracy test 0.5003170577045022
roc_auc_ test  0.501142096662115
Roc auc score test predict proba 0.502102168481902
 score  make_scorer(custom_score_func) : -338298.0
47.92938050031662 minutes


In [14]:
result_metier

,Algorithme,score nom,score,Meilleur param,best score,sampling,roc_auc_train,Roc_auc_train_discret,roc_auc_test,roc_auc_discret_test,TN,FP,FN,TP,minutes
0,LogisticRegression(random_state=22),make_scorer(custom_score_func),NaN,"{'C': 10, 'penalty': 'l2'}",9786.000000,FunctionSampler(func=<function nosamplerfunc a...,0.505484,0.748160,0.505475,0.749479,0.918161,0.001012,0.079851,0.000976,2.289600
0,LogisticRegression(random_state=22),make_scorer(custom_score_func),NaN,"{'C': 10, 'penalty': 'l2'}",-163063.000000,RandomOverSampler(random_state=22),0.683396,0.748433,0.684405,0.749661,0.633321,0.285852,0.026044,0.054783,4.989729
0,LogisticRegression(random_state=22),make_scorer(custom_score_func),NaN,"{'C': 10, 'penalty': 'l2'}",-14463.333333,RandomUnderSampler(random_state=22),0.682965,0.746622,0.682796,0.748575,0.628553,0.290621,0.025694,0.055132,0.309701
0,LogisticRegression(random_state=22),make_scorer(custom_score_func),NaN,"{'C': 0.1, 'penalty': 'l2'}",-128402.666667,SMOTETomek(random_state=22),0.677402,0.739145,0.675932,0.739239,0.642410,0.276763,0.027812,0.053015,51.364204
0,DecisionTreeClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 7, 'min_samples_leaf': 15}",9536.333333,FunctionSampler(func=<function nosamplerfunc a...,0.507657,0.729498,0.505813,0.718692,0.918364,0.000809,0.079518,0.001309,2.312820
0,DecisionTreeClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 5, 'min_samples_leaf': 5}",-148692.333333,RandomOverSampler(random_state=22),0.657808,0.715434,0.657061,0.712027,0.558360,0.360813,0.023589,0.057238,4.140918
0,DecisionTreeClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 7, 'min_samples_leaf': 15}",-15836.000000,RandomUnderSampler(random_state=22),0.668624,0.726789,0.661286,0.711456,0.610326,0.308848,0.026410,0.054417,0.375845
0,DecisionTreeClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 7, 'min_samples_leaf': 5}",-94697.666667,SMOTETomek(random_state=22),0.573967,0.665622,0.574272,0.660935,0.807144,0.112029,0.059018,0.021808,53.549543
0,RandomForestClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 3, 'n_estimators': 100}",9094.666667,FunctionSampler(func=<function nosamplerfunc a...,0.500000,0.716407,0.500000,0.717085,0.919173,0.000000,0.080827,0.000000,6.246915
0,RandomForestClassifier(random_state=22),make_scorer(custom_score_func),NaN,"{'max_depth': 5, 'n_estimators': 200}",-169492.666667,RandomOverSampler(random_state=22),0.667864,0.727161,0.660894,0.717949,0.603704,0.315469,0.025950,0.054876,13.808333


In [15]:
result_metier.to_excel(expe+".xls")

In [16]:
print ( time.time()-debut)

18228.617216825485
